In [1]:
pip install rdflib

In [2]:
import pandas as pd
import random
import csv
import os
from itertools import islice
import urllib.parse
from rdflib import Graph
from rdflib import Literal
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS, FOAF, XSD, URIRef

In [3]:
Abox = Namespace("http://SDM_LAB2.org/abox/")

g = Graph()
g.bind("abox", Abox)

In [4]:
# Read CSV
csv_files = [
    "abstracts-sample.csv",
    "affiliated-to.csv",
    "authors-sample.csv",
    "belongs-to.csv",
    "categoriesRelations.csv",
    "citations-sample.csv",
    "cited-by.csv",
    "companies.csv",
    "conferences.csv",
    "is-from.csv",
    "journals.csv",
    "keywords.csv",
    "paper-ids-sample.csv",
    "papers-processed.csv",
    "papers-sample.csv",
    "publication-venues-sample.csv",
    "published-in.csv",
    "related-to.csv",
    "reviewed-by.csv",
    "reviews.csv",
    "universities.csv",
    "volume-from.csv",
    "withAbstracts.csv",
    "written-by.csv"
]

csv_dir = '/content/drive/MyDrive/SDM_lab2/data/CSVs/'

csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

dataframes = {}
for csv_file in csv_files:
    file_path = os.path.join(csv_dir, csv_file)
    df_name = os.path.splitext(csv_file)[0].replace('-', '_')
    dataframes[df_name] = pd.read_csv(file_path)

for df_name in dataframes.keys():
    print(df_name)

paper_belong_to_proceeding
paper
conference
paper_presented_in_conference
paper_cite_paper
scopus_500
sample
journal
journal_in_year
authors
author_write
paper_belong_to_journal
conference_detail
author_review
paper_in_year
paper_has_keywords
year
proceeding
proceeding_in_year
keywords


# Load Class and Properties

## Load Class ResearchPaper

In [5]:
dataframes['paper'].head(3)

,title,abstract,pages,DOI,link,year
0,Challenges posed by hijacked journals in Scopus,This study presents and explains the phenomeno...,395-422,10.1002/asi.24855,https://www.scopus.com/inward/record.uri?eid=2...,2023
1,Normalizing Large Scale Sensor-Based MWD Data:...,In the context of geo-infrastructures and spec...,-,10.3390/s24041209,https://www.scopus.com/inward/record.uri?eid=2...,2022
2,Instant learning based on deep neural network ...,Biometric-based identity verification systems ...,32099-32122,10.1007/s11042-023-16751-6,https://www.scopus.com/inward/record.uri?eid=2...,2024


In [6]:
ResearchPaper = Abox.ResearchPaper

paper_properties = {
    "title": Abox.title,
    "abstract": Abox.abstract,
    "pages": Abox.pages,
    "DOI": Abox.DOI,
    "link": Abox.link,
    "year": Abox.year
}

def process_papers(df):
    for _, record in df.iterrows():
        paper_uri = URIRef(Abox["ResearchPaper_" + str(record['DOI'])])
        g.add((paper_uri, RDF.type, ResearchPaper))
        for key, prop in paper_properties.items():
            if pd.notna(record[key]):
                g.add((paper_uri, prop, Literal(record[key])))

process_papers(dataframes['paper'])

## Load Property of Cites

In [7]:
dataframes['paper_cite_paper'].head(3)

,start_id,end_id
0,10.1002/asi.24855,10.4230/LIPIcs.ICDT.2024.23
1,10.1002/asi.24855,10.1109/SSI58917.2023.10387755
2,10.1002/asi.24855,10.4230/LIPIcs.ICDT.2023.14


In [8]:
Cites = Abox.Cites

Cites_properties = {
    "start_id": Abox.start_id,
    "end_id": Abox.end_id
}

def process_paper(df):
    for _, record in df.iterrows():
        # Create URIs for the start and end papers
        start_paper_uri = URIRef(Abox["ResearchPaper_" + str(record['start_id'])])
        end_paper_uri = URIRef(Abox["ResearchPaper_" + str(record['end_id'])])

        # Add the type triples for both papers to the graph
        # g.add((start_paper_uri, RDF.type, ResearchPaper)) #Already Added Before
        # g.add((end_paper_uri, RDF.type, ResearchPaper))

        # Add the citation relationship to the graph
        g.add((start_paper_uri, Cites, end_paper_uri))

process_paper(dataframes['paper_cite_paper'])

## Load Class Topics

In [9]:
dataframes['keywords'].head(3)

,keywords
0,filtering process
1,measurement while drilling (MWD)
2,normalizing index


In [10]:
Topics = Abox.Topics  # Name in Abox

Topics_properties = {
    "keywords": Abox.topics
}

def process_topics(df):
    for _, record in df.iterrows():
        keywords = urllib.parse.quote("Topics_" + str(record['keywords']))  # URL Code
        keywords_uri = URIRef(Abox[keywords])
        g.add((keywords_uri, RDF.type, Topics))  #
        for key, prop in Topics_properties.items():
            if pd.notna(record[key]):
                g.add((keywords_uri, prop, Literal(record[key])))

process_topics(dataframes['keywords'])

## Load Property HasTopics

In [11]:
dataframes['paper_has_keywords'].head(3)

,paper_id,keywords
0,10.3390/s24041209,filtering process
1,10.3390/s24041209,measurement while drilling (MWD)
2,10.3390/s24041209,normalizing index


In [12]:
HasTopics = Abox.HasTopics

HasTopics_properties = {
    "paper_id": Abox.paper_id,
    "keywords": Abox.keywords
}

def process_paper(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(Abox["ResearchPaper_" + str(record['paper_id'])])
        keywords = urllib.parse.quote("Topics_" + str(record['keywords']))  # URL Code
        keywords_uri = URIRef(Abox[keywords])

        # Add the relationship to the graph
        g.add((paper_id_uri, HasTopics, keywords_uri))

process_paper(dataframes['paper_has_keywords'])

## Load Class SubmittedPaper

In [13]:
dataframes['paper'].head(3)

,title,abstract,pages,DOI,link,year
0,Challenges posed by hijacked journals in Scopus,This study presents and explains the phenomeno...,395-422,10.1002/asi.24855,https://www.scopus.com/inward/record.uri?eid=2...,2023
1,Normalizing Large Scale Sensor-Based MWD Data:...,In the context of geo-infrastructures and spec...,-,10.3390/s24041209,https://www.scopus.com/inward/record.uri?eid=2...,2022
2,Instant learning based on deep neural network ...,Biometric-based identity verification systems ...,32099-32122,10.1007/s11042-023-16751-6,https://www.scopus.com/inward/record.uri?eid=2...,2024


In [14]:
SubmittedPaper = Abox.SubmittedPaper

paper_properties = {
    "title": Abox.title,
    "abstract": Abox.abstract,
    "pages": Abox.pages,
    "DOI": Abox.DOI,
    "link": Abox.link,
    "year": Abox.year
}

def SubmittedPaper_process(df):
    for _, record in df.iterrows():
        paper_uri = URIRef(Abox["SubmittedPaper_" + str(record['DOI'])])
        g.add((paper_uri, RDF.type, SubmittedPaper))
        for key, prop in paper_properties.items():
            if pd.notna(record[key]):
                g.add((paper_uri, prop, Literal(record[key])))

SubmittedPaper_process(dataframes['paper'])

## Load Property Submitted

In [15]:
dataframes['paper'].head(3)

,title,abstract,pages,DOI,link,year
0,Challenges posed by hijacked journals in Scopus,This study presents and explains the phenomeno...,395-422,10.1002/asi.24855,https://www.scopus.com/inward/record.uri?eid=2...,2023
1,Normalizing Large Scale Sensor-Based MWD Data:...,In the context of geo-infrastructures and spec...,-,10.3390/s24041209,https://www.scopus.com/inward/record.uri?eid=2...,2022
2,Instant learning based on deep neural network ...,Biometric-based identity verification systems ...,32099-32122,10.1007/s11042-023-16751-6,https://www.scopus.com/inward/record.uri?eid=2...,2024


In [16]:
Submitted = Abox.Submitted

Submitted_properties = {
    "DOI": Abox.paper_id
}

def Submitted_process(df):
    for _, record in df.iterrows():
        # Create URIs
        paper_id_uri = URIRef(Abox["ResearchPaper_" + str(record['DOI'])])
        submitted_paper_id_uri = URIRef(Abox["SubmittedPaper_" + str(record['DOI'])])

        # Add the relationship to the graph
        g.add((paper_id_uri, Submitted, submitted_paper_id_uri))

Submitted_process(dataframes['paper'])

## Load Class Person

In [17]:
# The RDF should Automate Generate Person Accourding to Tbox

## Load Class Author

In [18]:
dataframes['authors'].head(3)

,author_id,author_name,author_affiliation
0,57219571200,Abalkina A.,"Freie Universität Berlin, Berlin, Germany"
1,35726817600,Abbaszadeh Shahri A.,"Johan Lundberg AB, Uppsala, 754 50, Sweden, D..."
2,58897160900,Shan C.,"Division of Rock Engineering, Tyrens, Stockho..."


In [19]:
Author = Abox.Author

Author_properties = {
    "author_id": Abox.title,
    "author_name": Abox.abstract,
    "author_affiliation": Abox.pages
}

def Author_process(df):
    for _, record in df.iterrows():
        Author_uri = URIRef(Abox["Author_" + str(record['author_id'])])
        g.add((Author_uri, RDF.type, Author))
        for key, prop in Author_properties.items():
            if pd.notna(record[key]):
                g.add((Author_uri, prop, Literal(record[key])))

Author_process(dataframes['authors'])

## Load Property Write

In [20]:
dataframes['author_write'].head(3)

,author_id,paper_id,corresponding
0,57219571200,10.1002/asi.24855,True
1,35726817600,10.3390/s24041209,True
2,58897160900,10.3390/s24041209,False


In [21]:
Write = Abox.Write

Write_properties = {
    "author_id": Abox.author_id,
    "paper_id": Abox.paper_id,
    "corresponding": Abox.corresponding
}

def Write_process(df):
    for _, record in df.iterrows():
        # Create URIs
        Author_id_uri = URIRef(Abox["Author_" + str(record['author_id'])])
        ResearchPaper_id_uri = URIRef(Abox["ResearchPaper_" + str(record['paper_id'])])

        # Add the relationship to the graph
        g.add((Author_id_uri, Write, ResearchPaper_id_uri))

Write_process(dataframes['author_write'])

## Load Property CorrespondingAuthor

In [22]:
dataframes['author_write'].head(3)

,author_id,paper_id,corresponding
0,57219571200,10.1002/asi.24855,True
1,35726817600,10.3390/s24041209,True
2,58897160900,10.3390/s24041209,False


In [23]:
CorrespondingAuthor = Abox.CorrespondingAuthor

Write_properties = {
    "author_id": Abox.author_id,
    "paper_id": Abox.paper_id,
    "corresponding": Abox.corresponding
}

def CorrespondingAuthor_process(df):
    for _, record in df.iterrows():
        # Create URIs
        Author_id_uri = URIRef(Abox["Author_" + str(record['author_id'])])
        ResearchPaper_id_uri = URIRef(Abox["ResearchPaper_" + str(record['paper_id'])])

        # If the corresponding field is True, add the CorrespondingAuthor relationship
        if record['corresponding']:
            g.add((Author_id_uri, CorrespondingAuthor, ResearchPaper_id_uri))

CorrespondingAuthor_process(dataframes['author_write'])

## Load Class Reviewer

In [24]:
dataframes['author_review'].head(3)

,paper_id,reviewer_id,review_content,accept_possibility
0,10.1002/asi.24855,55445000700,Next certain tend million country compare.,0.254872
1,10.1002/asi.24855,57946022500,Hold thus safe difficult focus.,0.466417
2,10.1002/asi.24855,25921550000,Drug fund sure political social.,0.122583


In [25]:
Reviewer = Abox.Reviewer

Reviewer_properties = {
    "paper_id": Abox.paper_id,
    "reviewer_id": Abox.reviewer_id,
    "review_content": Abox.review_content,
    "accept_possibility": Abox.accept_possibility
}

def Reviewer_process(df):
    for _, record in df.iterrows():
        Reviewer_uri = URIRef(Abox["Reviewer_" + str(record['reviewer_id'])])
        g.add((Reviewer_uri, RDF.type, Author))
        for key, prop in Reviewer_properties.items():
            if pd.notna(record[key]):
                g.add((Reviewer_uri, prop, Literal(record[key])))

Reviewer_process(dataframes['author_review'])

## Load Property WriteReview

In [26]:
dataframes['author_review'].head(3)

,paper_id,reviewer_id,review_content,accept_possibility
0,10.1002/asi.24855,55445000700,Next certain tend million country compare.,0.254872
1,10.1002/asi.24855,57946022500,Hold thus safe difficult focus.,0.466417
2,10.1002/asi.24855,25921550000,Drug fund sure political social.,0.122583


In [27]:
WriteReview = Abox.WriteReview

WriteReview_properties = {
    "paper_id": Abox.paper_id,
    "reviewer_id": Abox.reviewer_id,
    "review_content": Abox.review_content,
    "accept_possibility": Abox.accept_possibility
}

def WriteReview_process(df):
    for _, record in df.iterrows():
        # Create URIs
        Reviewer_id_uri = URIRef(Abox["Reviewer_" + str(record['reviewer_id'])])
        SubmittedPaper_id_uri = URIRef(Abox["SubmittedPaper_" + str(record['paper_id'])])

        # Add the relationship to the graph
        g.add((Reviewer_id_uri, WriteReview, SubmittedPaper_id_uri))

WriteReview_process(dataframes['author_review'])

## Load Class ReviewText

In [28]:
dataframes['author_review'].head(3)

,paper_id,reviewer_id,review_content,accept_possibility
0,10.1002/asi.24855,55445000700,Next certain tend million country compare.,0.254872
1,10.1002/asi.24855,57946022500,Hold thus safe difficult focus.,0.466417
2,10.1002/asi.24855,25921550000,Drug fund sure political social.,0.122583


In [29]:
ReviewText = Abox.ReviewText

ReviewText_properties = {
    "paper_id": Abox.paper_id,
    "reviewer_id": Abox.reviewer_id,
    "review_content": Abox.review_content,
    "accept_possibility": Abox.accept_possibility
}

def ReviewText_process(df):
    for _, record in df.iterrows():
        ReviewText = urllib.parse.quote("ReviewText_" + str(record['review_content']))  # URL Code
        ReviewText_uri = URIRef(Abox[ReviewText])
        g.add((ReviewText_uri, RDF.type, Author))
        for key, prop in ReviewText_properties.items():
            if pd.notna(record[key]):
                g.add((ReviewText_uri, prop, Literal(record[key])))

ReviewText_process(dataframes['author_review'])

## Load Property ReviewOf

In [30]:
dataframes['author_review'].head(3)

,paper_id,reviewer_id,review_content,accept_possibility
0,10.1002/asi.24855,55445000700,Next certain tend million country compare.,0.254872
1,10.1002/asi.24855,57946022500,Hold thus safe difficult focus.,0.466417
2,10.1002/asi.24855,25921550000,Drug fund sure political social.,0.122583


In [31]:
ReviewOf = Abox.ReviewOf

ReviewOf_properties = {
    "paper_id": Abox.paper_id,
    "reviewer_id": Abox.reviewer_id,
    "review_content": Abox.review_content,
    "accept_possibility": Abox.accept_possibility
}

def ReviewOf_process(df):
    for _, record in df.iterrows():
        # Create URIs
        ReviewText = urllib.parse.quote("ReviewText_" + str(record['review_content']))  # URL Code
        ReviewText_uri = URIRef(Abox[ReviewText])
        SubmittedPaper_id_uri = URIRef(Abox["SubmittedPaper_" + str(record['paper_id'])])

        # Add the relationship to the graph
        g.add((ReviewText_uri, ReviewOf, SubmittedPaper_id_uri))

ReviewOf_process(dataframes['author_review'])

## Load Property AcceptPossibility

In [ ]:
dataframes['author_review'].head(3)

In [ ]:
AcceptPossibility = Abox.AcceptPossibility

ReviewOf_properties = {
    "paper_id": Abox.paper_id,
    "reviewer_id": Abox.reviewer_id,
    "review_content": Abox.review_content,
    "accept_possibility": Abox.accept_possibility
}

def AcceptPossibility_process(df):
    for _, record in df.iterrows():
        # Create URIs
        ReviewText = urllib.parse.quote("ReviewText_" + str(record['review_content']))  # URL Code
        ReviewText_uri = URIRef(Abox[ReviewText])
        AcceptPossibility_id_uri = URIRef(Abox["AcceptPossibility_" + str(record['accept_possibility'])])

        # Add the relationship to the graph
        g.add((ReviewText_uri, AcceptPossibility, AcceptPossibility_id_uri))

AcceptPossibility_process(dataframes['author_review'])

## Load Class Journal

In [34]:
dataframes['journal'].head(3)

,journal_name
0,VLDB Journal
1,"Circuits, Systems, and Signal Processing"
2,Computer Methods and Programs in Biomedicine


In [38]:
Journal = Abox.Journal

Journal_properties = {
    "journal_name": Abox.journal_name
}

def Journal_process(df):
    for _, record in df.iterrows():
        Journal = urllib.parse.quote("Journal_" + str(record['journal_name']))  # URL Code
        Journal_uri = URIRef(Abox[ReviewText])
        g.add((Journal_uri, RDF.type, Author))
        for key, prop in Journal_properties.items():
            if pd.notna(record[key]):
                g.add((Journal_uri, prop, Literal(record[key])))

Journal_process(dataframes['journal'])

# Export graph

In [32]:
for stmt in islice(g, 5):
    print(stmt)

(rdflib.term.URIRef('http://SDM_LAB2.org/abox/Topics_Data%20Systems%20Education'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://SDM_LAB2.org/abox/Topics'))
(rdflib.term.URIRef('http://SDM_LAB2.org/abox/Reviewer_7801327466'), rdflib.term.URIRef('http://SDM_LAB2.org/abox/reviewer_id'), rdflib.term.Literal('7801327466', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://SDM_LAB2.org/abox/ResearchPaper_10.1145/3555776.3577733'), rdflib.term.URIRef('http://SDM_LAB2.org/abox/Cites'), rdflib.term.URIRef('http://SDM_LAB2.org/abox/ResearchPaper_10.1145/3583120.3586957'))
(rdflib.term.URIRef('http://SDM_LAB2.org/abox/SubmittedPaper_10.30420/456164024'), rdflib.term.URIRef('http://SDM_LAB2.org/abox/abstract'), rdflib.term.Literal('There is an emerging need for comparable data for multi-microphone processing, particularly in acoustic sensor networks. However, commonly available databases are oft

In [33]:
# Export as Turtle
g.serialize(destination='output.ttl', format='turtle')

<Graph identifier=Nb7425494545b4226946b8c22e9721b13 (<class 'rdflib.graph.Graph'>)>